In [2]:
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import numpy as np
import scipy.io as sio
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import random
from collections import Counter
np.random.seed(0)
random.seed(0)
ran=0
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn import preprocessing

from sklearn.svm import SVC

from sklearn.decomposition import PCA

from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import os
import seaborn as sn
import pandas as pd
from sklearn.utils import class_weight
from sklearn import preprocessing

In [3]:

path = r"D:\COVID\result\all_compare\mat"

Data_ver=['COVID','healthy','dyspnea_normal','dyspnea_exercise','sleep'] 
# Join various path components
path_t=os.path.join(path, Data_ver[0]+'_feat.mat')
data = sio.loadmat(path_t)
feat_acc_covid=data['EpochFeat_sel_acc_all_p'] # 3 optimal channels, choose 0 channel first  
feat_ncs_covid=data['EpochFeat_sel_ncs_all_p']
p_ncs_covid=data['p_num_ncs_all_p']  # number of participant of every feature 
p_acc_covid=data['p_num_acc_all_p']

path_t=os.path.join(path, Data_ver[1]+'_feat.mat')
data = sio.loadmat(path_t)
feat_acc_healthy=data['EpochFeat_sel_acc_all_p'] # 3 optimal channels, choose 0 channel first  
feat_ncs_healthy=data['EpochFeat_sel_ncs_all_p']
p_ncs_healthy=data['p_num_ncs_all_p']  # number of participant of every feature 
p_acc_healthy=data['p_num_acc_all_p']

path_t=os.path.join(path, Data_ver[2]+'_feat.mat')
data = sio.loadmat(path_t)
feat_ncs_dysp_Normal=data['EpochFeat_ncs_all'] # 3 optimal channels, choose 0 channel first  
feat_bio_dysp_Normal=data['EpochFeat_bio_all']
p_ncs_dysp_Normal=data['case_num_ncs']  # number of participant of every feature 
p_acc_dysp_Normal=data['case_num_ncs']

path_t=os.path.join(path, Data_ver[3]+'_feat.mat')
data = sio.loadmat(path_t)
feat_ncs_dysp_Exercise=data['EpochFeat_ncs_all'] 
feat_bio_dysp_Exercise=data['EpochFeat_bio_all']
p_ncs_dysp_Exercise=data['case_num_ncs'] 
p_acc_dysp_Exercise=data['case_num_ncs']

path_t=os.path.join(path, Data_ver[4]+'_feat.mat')
data = sio.loadmat(path_t)
feat_ncs_sleep_Normal=data['featureNCS_0']
feat_ncs_sleep_Disorder=data['featureNCS_1']
p_ncs_sleep_Normal=data['caseNumNCS_0']
p_ncs_sleep_Disorder=data['caseNumNCS_1']

feat_psg_sleep_Normal=data['featurePSG_0']
feat_psg_sleep_Disorder=data['featurePSG_1']
p_psg_sleep_Normal=data['caseNumPSG_0']
p_psg_sleep_Disorder=data['caseNumPSG_1']



# model 1: binary class  only label normal =1, exercise =0  in dyspnea study 
label_ncs_dysp_Exercise=np.zeros((np.shape(feat_ncs_dysp_Exercise)[0],1))
label_ncs_dysp_Exercise[:]=1

label_ncs_dysp_Normal=np.zeros((np.shape(feat_ncs_dysp_Normal)[0],1))
label_ncs_dysp_Normal[:]=0

In [4]:
X_train=np.vstack((feat_ncs_dysp_Normal,feat_ncs_dysp_Exercise))
y_train=np.vstack((label_ncs_dysp_Normal,label_ncs_dysp_Exercise)).ravel()

X_test=feat_acc_covid[:,:51,0]
y_test=np.zeros((np.shape(p_acc_covid)[0],1))
y_test[:]=1

y_test=y_test.ravel()

# X_train=X_train[:,:37]
# X_test=X_test[:,:37]
X_train_scaler=preprocessing.StandardScaler().fit_transform(X_train)
X_test_scaler=preprocessing.StandardScaler().fit_transform(X_test)

X_test2=feat_acc_healthy[:,:51,0]


In [7]:

clf1=KNeighborsClassifier(n_neighbors=40)

clf2 = DecisionTreeClassifier(random_state=0,max_depth=20)

clf3 = GaussianNB(var_smoothing=0.5)
clf4= svm.SVC(kernel="rbf", C=0.2)
clf5 = RandomForestClassifier(max_depth=13,random_state=0,n_estimators=3)
clf6 = BaggingClassifier(base_estimator=clf3,n_estimators=10, random_state=0)

clf7 = xgb.XGBClassifier(n_estimators=5)
 
# clf=clf7.fit(X_train, y_train)
# y_pred_test = clf.predict(X_test)
model=clf1.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

y_pred_test=np.array(y_pred_test)

mean_y=np.mean(y_pred_test)
print(mean_y)
print(np.where(y_pred_test==0))

y_pred_test2 = model.predict(X_test2)
y_pred_test2=np.array(y_pred_test2)
mean_y2=np.mean(y_pred_test2)
print(mean_y2)

pred_psg_sleep_Normal = model.predict(feat_psg_sleep_Normal)
pred_psg_sleep_Normal=np.array(pred_psg_sleep_Normal)
mean_pred_psg_sleep_Normal=np.mean(pred_psg_sleep_Normal)
print(mean_pred_psg_sleep_Normal)

pred_psg_sleep_Disorder = model.predict(feat_psg_sleep_Disorder)
pred_psg_sleep_Disorder=np.array(pred_psg_sleep_Disorder)
mean_pred_psg_sleep_Disorder=np.mean(pred_psg_sleep_Disorder)
print(mean_pred_psg_sleep_Disorder)


1.0
(array([], dtype=int64),)
0.33663366336633666
0.05520940564555453
0.0568015602145295


In [177]:
# clf1 = LogisticRegression(random_state=0,C=0.7,max_iter=10000) not good
clf2 = DecisionTreeClassifier(random_state=0,max_depth=13)

clf3 = GaussianNB(var_smoothing=0.5)
clf4= svm.SVC(kernel="rbf", C=0.3,probability=True)
clf5 = RandomForestClassifier(max_depth=13,random_state=0,n_estimators=3)
clf6 = BaggingClassifier(base_estimator=clf3,n_estimators=10, random_state=0)

clf7 = xgb.XGBClassifier(n_estimators=100)
 
# clf=clf7.fit(X_train, y_train)
# y_pred_test = clf.predict(X_test)
model=clf3.fit(X_train, y_train)
y_pred_test = model.predict_proba(X_test)

y_pred_test=np.array(y_pred_test)

mean_y=np.mean(y_pred_test[:,1])
print(mean_y)

y_pred_test2 = model.predict_proba(X_test2)

y_pred_test2=np.array(y_pred_test2)

mean_y2=np.mean(y_pred_test2[:,1])
print(mean_y2)



0.8623716151179771
0.4116567565398503
